<a href="https://colab.research.google.com/github/JAYESH1304/projects/blob/main/TunedBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 28.1 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys
import re
import html
import string

In [4]:
!git clone https://github.com/sydney-machine-learning/COVID19_sentinentanalysis

Cloning into 'COVID19_sentinentanalysis'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 309 (delta 61), reused 134 (delta 52), pack-reused 165
Receiving objects: 100% (309/309), 93.07 MiB | 28.05 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [5]:
train = pd.read_csv("/content/drive/MyDrive/labeledEn.csv")
test = pd.read_csv("/content/COVID19_sentinentanalysis/data/ieee_extract.csv")

In [6]:
train.columns

Index(['ID', 'Tweet', 'Optimistic', 'Thankful', 'Empathetic', 'Pessimistic',
       'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise',
       'Joking'],
      dtype='object')

In [ ]:
train = train.drop(columns=['ID'],axis=1)

In [ ]:
labels = ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic',
       'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise',
       'Joking']

In [ ]:
def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))

def to_lowercase(text):
    return text.lower()

def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)

def remove_whitespaces(text):
    return text.strip()

def clean_text( text):
    text = remove_special_chars(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)

    return ''.join(text)

In [ ]:
train['Tweet'] = train['Tweet'].apply(lambda x: clean_text(x))

In [ ]:
train['Tweet']

0       a glass of wine keeps the corona away drake no...
1       can anyone tell me if you took the flu shot la...
2       btw producers send me beats im working on musi...
3       when someone you know apart of your family die...
4       dear soccer i really miss you please come back...
                              ...                        
9995    one good thing about the quarantine i can now ...
9996    shoutout to for making the coronavirus testing...
9997    i find it amazing that more people fear the co...
9998    thinking about how i put my two weeks in right...
9999    stop asking to change location manthat how you...
Name: Tweet, Length: 10000, dtype: object

In [ ]:
MAX_LEN = 256
train_batch_size = 16
valid_batch_size = 16
EPOCHS = 2
Learning_rate = 1e-05

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
   def __init__(self,df,tokenizer,max_len):
     self.df = df
     self.tokenizer = tokenizer
     self.max_len = max_len
     self.title = self.df['Tweet']
     self.targets = self.df[labels].values

   def __len__(self):
     return len(self.title)
    
   def __getitem__(self,index):
     title = str(self.title[index])
     title = " ".join(title.split())

     inputs = self.tokenizer.encode_plus(
         title,
         None,
         add_special_tokens=True,
         max_length=self.max_len,
         padding='max_length',
         return_token_type_ids=True,
         truncation=True,
         return_attention_mask=True,
         return_tensors='pt'
     )
     return {
         'input_ids':inputs['input_ids'].flatten(),
         'attention_mask':inputs['attention_mask'].flatten(),
         'token_type_ids':inputs['token_type_ids'].flatten(),
         'targets':torch.FloatTensor(self.targets[index])
     }

In [ ]:
train_size=0.8
train_df = train.sample(frac=train_size, random_state=200).reset_index(drop=True)
val = train.drop(train_df.index).reset_index(drop=True)

In [ ]:
train_dataset = CustomDataset(train_df,tokenizer,MAX_LEN)
valid_dataset = CustomDataset(val,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=train_batch_size,
    num_workers=0
)
val_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    shuffle=False,
    batch_size=valid_batch_size,
    num_workers=0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
  checkpoint = torch.load(checkpoint_fpath)
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  valid_loss_min = checkpoint['valid_loss_min']
  return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
  f_path = checkpoint_path
  torch.save(state, f_path)
  if is_best:
    best_fpath = best_model_path
    shutil.copyfile(f_path, best_fpath)

In [ ]:
 class BERTClass(nn.Module):
   def __init__(self):
     super(BERTClass, self).__init__()
     self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
     self.dropout = nn.Dropout(0.3)
     self.linear = nn.Linear(768, 11)

   def forward(self, input_ids, attention_mask, token_type_ids):
     output = self.bert_model(input_ids, attention_mask, token_type_ids)
     output_dropout = self.dropout(output.pooler_output)
     output = self.linear(output_dropout)
     return output

model = BERTClass()
model.to(device)   


In [ ]:
def loss_fn(outputs, targets):
  return nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params=model.parameters(), lr=Learning_rate)  

In [ ]:
def train_model(
    n_epochs,
    training_loader,
    validation_loader,
    model,
    optimizer,
    checkpoint_path,
    best_model_path
):
   valid_loss_min = np.Inf

   for epoch in range(1, n_epochs+1):
     train_loss=0
     valid_loss=0
     model.train()
     #trining loop
     for index, batch in enumerate(training_loader):
       input_ids = batch['input_ids'].to(device, dtype=torch.long)
       attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
       token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
       targets =  batch['targets'].to(device, dtype=torch.float)
       outputs = model(input_ids, attention_mask, token_type_ids)
       optimizer.zero_grad()
       loss = loss_fn(outputs, targets)
       optimizer.zero_grad()
       loss.backward()
       optimizer.step()
       train_loss = train_loss + (1/((index+1)*(loss.item()-train_loss)))

     #validation loop
     model.eval()
     with torch.no_grad():
       for index, batch in enumerate(validation_loader):
         input_ids = batch['input_ids'].to(device, dtype=torch.long)
         attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
         token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
         targets =  batch['targets'].to(device, dtype=torch.float)
         outputs = model(input_ids, attention_mask, token_type_ids)
         loss = loss_fn(outputs, targets)
         valid_loss = valid_loss + (1/((index+1)*(loss.item()-valid_loss)))

     checkpoint = {
         'epoch': epoch+1,
         'valid_loss_min':valid_loss,
         'state_dict':model.state_dict(),
         'optimizer':optimizer.state_dict()
     }
     save_ckp(checkpoint, False, checkpoint_path, best_model_path)

   return model

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/curr_ckpt",'/best.pt' )

In [ ]:
text = "fuck you covid , you are son of bitch"
text

'fuck you covid , you are son of bitch'

In [ ]:
inputs = tokenizer.encode_plus(
         text,
         None,
         add_special_tokens=True,
         max_length=MAX_LEN,
         padding='max_length',
         return_token_type_ids=True,
         truncation=True,
         return_attention_mask=True,
         return_tensors='pt'
     )

trained_model.eval()
with torch.no_grad():
  input_ids = inputs['input_ids'].to(device, dtype=torch.long)
  attention_mask = inputs['attention_mask'].to(device, dtype=torch.long)
  token_type_ids = inputs['token_type_ids'].to(device, dtype=torch.long)
  output = trained_model(input_ids, attention_mask, token_type_ids)
  final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
  print(final_output)



[[0.07842353731393814, 0.04523595795035362, 0.035178735852241516, 0.11295824497938156, 0.1311095803976059, 0.24325032532215118, 0.8441055417060852, 0.07769236713647842, 0.0817243754863739, 0.11818902939558029, 0.20882660150527954]]


In [ ]:
torch.save(trained_model,'/content/drive/MyDrive/saved models/TunedBert.pth')